In [27]:
import pandas as pd
import mysql.connector
import os

In [31]:
csv_files = csv_files = [
    ('airlines_cleaned.csv', 'airlines_cleaned'),
    ('airports_cleaned.csv', 'airports_cleaned'),
    ('cancellation_codes_cleaned.csv', 'cancellation_codes_cleaned'),
    ('flights_cleaned.csv', 'flights_cleaned') # Added payments.csv for specific handling
]

In [32]:
conn = mysql.connector.connect(
    host='LAPTOP-HSKVBD5P',
    user='bhuvan',
    password='Bhuvan@0128',
    database='airlines'
)
cursor = conn.cursor()

In [33]:
folder_path = "C:/Users/RAMESH/OneDrive/Desktop/airlines foder"

In [34]:
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing airlines_cleaned.csv
NaN values before replacement:
IATA_CODE    0
AIRLINE      0
dtype: int64

Processing airports_cleaned.csv
NaN values before replacement:
IATA_CODE    0
AIRPORT      0
CITY         0
STATE        0
COUNTRY      0
LATITUDE     3
LONGITUDE    3
dtype: int64

Processing cancellation_codes_cleaned.csv
NaN values before replacement:
CANCELLATION_REASON         0
CANCELLATION_DESCRIPTION    0
dtype: int64

Processing flights_cleaned.csv
NaN values before replacement:
YEAR                       0
MONTH                      0
DAY                        0
DAY_OF_WEEK                0
AIRLINE                    0
FLIGHT_NUMBER              0
TAIL_NUMBER               59
ORIGIN_AIRPORT             0
DESTINATION_AIRPORT        0
SCHEDULED_DEPARTURE        0
DEPARTURE_TIME          1104
DEPARTURE_DELAY         1104
TAXI_OUT                1127
WHEELS_OFF              1127
SCHEDULED_TIME             0
ELAPSED_TIME            1247
AIR_TIME                1247
DISTANCE 